<a href="https://colab.research.google.com/github/ArchoCode/colabTest/blob/main/simpleClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install emojis package

In [82]:
!pip install -U -q emoji

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as sm
%matplotlib inline
import re
import emoji
import nltk
nltk.download('punkt')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [84]:
data_set= pd.read_csv('/content/tweets.csv')

In [85]:
data_set.head()

,0,Comment,label
0,1,නැති බැරි කම් අග හිග කම් මැද ජීවත් වෙනවට වඩා හ...,1
1,2,ඔබ සැමට සුභ දවසක් වේවා!,0
2,3,හැමදේම අත හැර ගියා නම් හරි වගේ,1
3,4,රෑට දිලිසෙන ලුමිනස් පාට තරු ටිකක් කාමරේ ඇලෙව්ව...,0
4,5,මට මානසික ගැටලුවක් තියෙනවා. කතාකරන්න පුලුවන් ක...,1


Text cleaning of the dataset

In [86]:
def clean_text(df, text_field,new_text_field):
    df[new_text_field]=df[text_field]
    # remove numbers
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r"\d+", "", elem))
    #remove url
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r"https?://\S+|www\.\S+", "", elem))
    #remove HTML tags
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r"<.*?>", "", elem))
    #remove emojis 
    df[new_text_field] = df[new_text_field].apply(lambda elem: emoji.get_emoji_regexp().sub(r" ", elem))
    #remove re-tweets status 
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r'^RT @\w*: ', ' ', elem))
    #remove mentions 
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r'@\w*', ' ', elem))
    #remove special characters 
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r'[!@#&*$.?,]', ' ', elem))
    #remove \n
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub(r'\n', ' ', elem))
    #remove ''
    df[new_text_field] = df[new_text_field].apply(lambda elem: re.sub("'", '', elem))
    
    return df

#tokenizing
def tokenize(df,text_field,new_text_field):
  df[new_text_field]=df[text_field]
  df[new_text_field]=df[new_text_field].apply(word_tokenize)
  return df

clean_data = clean_text(data_set,'Comment','text_clean')
tokenize_data=tokenize(data_set,'text_clean','tokenized_text')
clean_data.head()

,0,Comment,label,text_clean,tokenized_text
0,1,නැති බැරි කම් අග හිග කම් මැද ජීවත් වෙනවට වඩා හ...,1,නැති බැරි කම් අග හිග කම් මැද ජීවත් වෙනවට වඩා හ...,"[නැති, බැරි, කම්, අග, හිග, කම්, මැද, ජීවත්, වෙ..."
1,2,ඔබ සැමට සුභ දවසක් වේවා!,0,ඔබ සැමට සුභ දවසක් වේවා,"[ඔබ, සැමට, සුභ, දවසක්, වේවා]"
2,3,හැමදේම අත හැර ගියා නම් හරි වගේ,1,හැමදේම අත හැර ගියා නම් හරි වගේ,"[හැමදේම, අත, හැර, ගියා, නම්, හරි, වගේ]"
3,4,රෑට දිලිසෙන ලුමිනස් පාට තරු ටිකක් කාමරේ ඇලෙව්ව...,0,රෑට දිලිසෙන ලුමිනස් පාට තරු ටිකක් කාමරේ ඇලෙව්ව...,"[රෑට, දිලිසෙන, ලුමිනස්, පාට, තරු, ටිකක්, කාමරේ..."
4,5,මට මානසික ගැටලුවක් තියෙනවා. කතාකරන්න පුලුවන් ක...,1,මට මානසික ගැටලුවක් තියෙනවා කතාකරන්න පුලුවන් ක...,"[මට, මානසික, ගැටලුවක්, තියෙනවා, කතාකරන්න, පුලු..."


Test & Train Split

In [87]:
# extract the labels from the train data
y = data_set.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(data_set.text_clean.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

Vectorize tweets using CountVectorizer

In [88]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True)

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

Model building

SVM classifier

In [90]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

Accuracy score

In [91]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  65.75342465753424 %


In [92]:
def classify_tweet(arr):
  return svm.predict(vectorizer.transform(arr))

In [94]:
msg=["මම සතුටින් ඉන්නෙ"]
for index_instance,instance in enumerate(classify_tweet(msg)):
  print(msg[index_instance], ' - ',instance)

මම සතුටින් ඉන්නෙ  -  0


මම සතුටින් ඉන්නෙ
මට මානසික ගැටලුවක් තියෙනවා